In [1]:
import pandas as pd
#df = pd.read_csv('datasets/erroneuous_test_data_set.csv')
df = pd.read_csv('datasets/adult.csv')


In [6]:
df2 = df[:10]
print(df2)
df2.to_excel('datasets/synthetic_data_adult.xlsx')

   age         workclass  fnlwgt     education  educational-num  \
0   25           Private  226802          11th                7   
1   38           Private   89814       HS-grad                9   
2   28         Local-gov  336951    Assoc-acdm               12   
3   44           Private  160323  Some-college               10   
4   18                 ?  103497  Some-college               10   
5   34           Private  198693          10th                6   
6   29                 ?  227026       HS-grad                9   
7   63  Self-emp-not-inc  104626   Prof-school               15   
8   24           Private  369667  Some-college               10   
9   55           Private  104996       7th-8th                4   

       marital-status         occupation   relationship   race  gender  \
0       Never-married  Machine-op-inspct      Own-child  Black    Male   
1  Married-civ-spouse    Farming-fishing        Husband  White    Male   
2  Married-civ-spouse    Protective-serv

In [46]:
import pandas as pd
import numpy as np
import random
# suppose your existing dataframe is df
dataset = pd.read_csv('datasets/adult.csv')# replace with your file
df = dataset[:50]


#generate missingness in 10% of data
num_missing = int(0.1* len(df))
missing_positions = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1])]
for row, col in random.sample(missing_positions, num_missing):
    df.iat[row, col] = np.nan if random.random() > 0.5 else None

#generate random variations in categorical string oclumns
for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].apply(lambda x: x if (random.random() > 0.05 or pd.isnull(x)) else x + random.choice(['VAR1', 'var2', 'V@R3', '4','five',]))

# Creating a column with mixed data types
df['mixed_data_types'] = pd.concat([pd.Series(np.random.choice([1, 2, 3], df.shape[0]//2)), pd.Series(np.random.choice(['MD1', 'MD2', 'MD3'], df.shape[0] - df.shape[0]//2))]).sample(frac=1).reset_index(drop=True)

# Adding special characters
random_col = random.choice(df.columns)
special_char_col = df[random_col].sample(frac=0.1, replace=True) # 10% of rows
special_char_col = special_char_col.apply(lambda x: random.choice(['@#$@', '###']) if isinstance(x, str) else x)
df.update(special_char_col)

# Creating conflicting labels
conflict_df = df.sample(frac=0.1, replace=True) # 10% of rows
conflict_df['income'] = conflict_df['income'].apply(lambda x: '<=50K' if x=='>50K' else '>50K')
df.update(conflict_df)

# Creating class imbalance
df['income'] = df['income'].apply(lambda x: x if random.random() > 0.8 else ('<=50K' if x=='>50K' else '>50K')) # 80-20 imbalance


#replace some rows for duplicates
to_replace = df.sample(frac=0.1).index
duplicate_rows_df = df.sample(n=len(to_replace), replace=True)
duplicate_rows_df.index = to_replace

# Replace the rows
df.loc[to_replace] = duplicate_rows_df

# Duplicate the column
random_col = random.choice(df.columns)
df[random_col + '2'] = df[random_col]

# Creating a column with a single unique value (redundant column)
df['redundant_SV_column'] = 'this_is_the_same'


df = df.reset_index(drop=True)



C:\Users\robin\AppData\Local\Temp\ipykernel_3432\142255163.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x if (random.random() > 0.05 or pd.isnull(x)) else x + random.choice(['VAR1', 'var2', 'V@R3', '4','five',]))
C:\Users\robin\AppData\Local\Temp\ipykernel_3432\142255163.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x if (random.random() > 0.05 or pd.isnull(x)) else x + random.choice(['VAR1', 'var2', 'V@R3', '4','five',]))
C:

In [47]:
print(df)
print(len(df))
df.to_excel('datasets/synthetic_test_dataset.xlsx')

    age         workclass    fnlwgt         education  educational-num  \
0    25           Private  226802.0              11th                7   
1    38           Private   89814.0           HS-grad                9   
2    28         Local-gov  336951.0        Assoc-acdm               12   
3    44           Private  160323.0      Some-college               10   
4    18                 ?  103497.0      Some-college               10   
5    34           Private  198693.0              10th                6   
6    29                 ?  227026.0           HS-grad                9   
7    63  Self-emp-not-inc  104626.0       Prof-school               15   
8    24           Private  369667.0      Some-college               10   
9    55           Private  104996.0           7th-8th                4   
10   65           Private  184454.0           HS-grad                9   
11   36       Federal-gov  212465.0         Bachelors               13   
12   26           Private       NaN   